<a href="https://colab.research.google.com/github/Mugheerik/cnn_project/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALLING REQUIRED LIBRARIES**

In [ ]:
pip install matplotlib tensorflow


**IMPORTING THE LIBRARIES**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


**LOAD AND PREPORCESS DATA SET**

In [ ]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# One-hot encode labels
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)


170498071/170498071 [==============================] - 6s 0us/step


**DATA AUGMENTATION**

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)

datagen.fit(train_images)


**CNN MODEL**

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


**TRAIN MODEL**

In [ ]:
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) / 32, epochs=10,
                    validation_data=(test_images, test_labels))


Epoch 1/10
1562/1562 [==============================] - 122s 78ms/step - loss: 1.5262 - accuracy: 0.4446 - val_loss: 1.2505 - val_accuracy: 0.5520
Epoch 2/10
1562/1562 [==============================] - 127s 81ms/step - loss: 1.3030 - accuracy: 0.5315 - val_loss: 1.1887 - val_accuracy: 0.5759
Epoch 3/10
1562/1562 [==============================] - 119s 76ms/step - loss: 1.2062 - accuracy: 0.5731 - val_loss: 1.2104 - val_accuracy: 0.5848
Epoch 4/10
1562/1562 [==============================] - 118s 75ms/step - loss: 1.1308 - accuracy: 0.6012 - val_loss: 1.0478 - val_accuracy: 0.6350
Epoch 5/10
1562/1562 [==============================] - 116s 74ms/step - loss: 1.0785 - accuracy: 0.6201 - val_loss: 0.9948 - val_accuracy: 0.6585
Epoch 6/10
1562/1562 [==============================] - 113s 72ms/step - loss: 1.0388 - accuracy: 0.6336 - val_loss: 0.9634 - val_accuracy: 0.6600
Epoch 7/10
1562/1562 [==============================] - 113s 72ms/step - loss: 1.0079 - accuracy: 0.6457 - val_loss: 0

**EVALUATION**

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test accuracy: {test_acc}')


313/313 - 4s - loss: 0.9211 - accuracy: 0.6849 - 4s/epoch - 12ms/step
Test accuracy: 0.6848999857902527


In [ ]:
model.save('cifar10_model.h4')

In [ ]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# List the contents of the current directory
directory_contents = os.listdir(current_directory)
print("Directory Contents:", directory_contents)


Current Working Directory: /content
Directory Contents: ['.config', 'cifar10_model.h4', 'sample_data']


# **Deploy**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
from io import BytesIO
import requests
import numpy as np

# Load the model
model = load_model('cifar10_model.h4')

# Map class index to labels
class_labels = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

def preprocess_image_from_url(image_url):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    img = img.resize((32, 32))  # Resize to match the model's expected input size
    img_array = np.asarray(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array


In [ ]:
image_url = 'https://th.bing.com/th/id/OIP.ePdSFdoz-QuuBqSPfmAP_AHaE6?rs=1&pid=ImgDetMain'

In [ ]:
# Preprocess the image
input_image = preprocess_image_from_url(image_url)

# Make predictions
predictions = model.predict(input_image)
predicted_class = np.argmax(predictions[0])
predicted_label = class_labels[predicted_class]

print(f'The predicted class is: {predicted_label}')


1/1 [==============================] - 0s 143ms/step
The predicted class is: frog
